# Heterologous pathway addition and importing the model

## importing the model

In [1]:
import code.io as io
model = io.read_yeast_model() # loading
from cobra import Reaction, Metabolite, Gene

In [2]:
model

Name,yeastGEM_v8__46__7__46__0
Memory address,11796d0d0
Number of metabolites,2807
Number of reactions,4133
Number of genes,1163
Number of groups,106
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


Since there are no production pathways inherently in the model of the organism we will need to add the metabolite and reaction ourselves

Adding the metabolites:

In [4]:
Beta_farnesene = Metabolite(name='Beta-farnesene',id='beta_farnesene_c',compartment='c')
# Created the metabolite Beta-farnesene as a metabolite that is present in the cytoplasm

Adding the reaction for beta-farnesene and the demand reaction for beta-farnesene

In [6]:
from cobra import Reaction, Metabolite, Gene

Beta_farnesene_synthase_reaction = Reaction('R08695') # Creates the Beta-farnesene reaction 

# This creates the reaction for beta-farnesene synthesis by indicating the consumption of the substrates and 
# using our newly introduced metabolite as the product
# The reaction R08695 is:
# Trans,trans-Farnesyl diphosphate <=> beta-Farnesene + Diphosphate
# These are found as the metabolites:
# Farnesyl diphosphate = s_0190
# beta-farnesene = Beta_farnesene
# Diphosphate = s_0633 

# Thus having already defined the metabolite and the reaction we can fully define the reaction stoichiometry:

Beta_farnesene_synthase_reaction.add_metabolites({model.metabolites.s_0190: -1,
                                                  model.metabolites.s_0633: 1,
                                                  Beta_farnesene: 1 # our product
                                                  }) 

# Test if the reaction has been made properly and the stoichiometric definition is correct:
Beta_farnesene_synthase_reaction

# Adding the reaction to the model
model.add_reactions([Beta_farnesene_synthase_reaction])

# tests that the reaction has been added to the model
model.reactions.get_by_id(Beta_farnesene_synthase_reaction.id)

Ignoring reaction 'R08695' since it already exists.


Reaction identifier,R08695
Name,
Memory address,0x28a04b550
Stoichiometry,s_0190 --> beta_farnesene_c + s_0633 farnesyl diphosphate --> Beta-farnesene + diphosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
# This creates the demand reaction for beta-farnesene as this otherwise would become a dead-end metabolite:

# The demand reaction follows the same framework as above, with the stoichiometry just being -1 for the
# metabolite Beta-farnesene:

Beta_farnesene_demand_reaction = Reaction('BFD')
Beta_farnesene_demand_reaction.add_metabolites({
                                                  Beta_farnesene: -1 # our product
                                                  })
model.add_reactions([Beta_farnesene_demand_reaction])

# Note: the output is because we already added the reaction to the model.

Ignoring reaction 'BFD' since it already exists.


In [10]:
# tests that the reaction has been added to the model
model.reactions.get_by_id(Beta_farnesene_demand_reaction.id)



Reaction identifier,BFD
Name,
Memory address,0x28a04be50
Stoichiometry,beta_farnesene_c --> Beta-farnesene -->
GPR,
Lower bound,0.0
Upper bound,1000.0


## Testing to make sure flux goes through the production pathway that we have created

In [15]:
# This function test to see that the organism still is growing
with model:
   print(model.optimize().objective_value)

# Now we will test to see if any flux is going through 


0.0819281723418478


# Saving the model:

In [ ]:
# This code saves the changes to the model and we now have a model that has a the heterologous pathway for 
# beta-farnesene production as well as a demand reaction for cytoplasmic beta-farnesene

io.write_yeast_model(model)   # saving

# Finding the most suitable carbon source and testing different types of media

First we must test that there